In [97]:
import http.client
import config
import pandas as pd
import json

In [ ]:
def get_arrivals(airlines, dates):
    conn = http.client.HTTPSConnection("flight-fare-search.p.rapidapi.com")
    conn2 = http.client.HTTPSConnection("flight-fare-search.p.rapidapi.com")

    headers = {
        'X-RapidAPI-Key': f"{config.api_key}",
        'X-RapidAPI-Host': "flight-fare-search.p.rapidapi.com"
    }

    df_arrivals = pd.DataFrame()  # Initialize an empty DataFrame
    df_departures = pd.DataFrame()  # Initialize an empty DataFrame

    for airline in airlines:
        for date in dates:
            conn.request("GET", f"/v2/airport/arrivals?airportCode=ATL&date={date}&carrierCode={airline}", headers=headers)
            conn2.request("GET", f"/v2/airport/departures?airportCode=ATL&date={date}&carrierCode={airline}", headers=headers)

            arrivals = conn.getresponse().read().decode('utf-8')
            departures = conn2.getresponse().read().decode('utf-8')

            arrivals = json.loads(arrivals)
            departures = json.loads(departures)

            arrivals_data = arrivals['result']['flights']
            departures_data = departures['result']['flights']

            df_arrivals_temp = pd.json_normalize(arrivals_data)  # Temporary DataFrame for each request
            df_departures_temp = pd.json_normalize(departures_data)  # Temporary DataFrame for each request

            df_arrivals = pd.concat([df_arrivals, df_arrivals_temp])  # Concatenate the temporary DataFrame with the main one

    return df_arrivals, df_departures

In [108]:
conn = http.client.HTTPSConnection("flight-fare-search.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': f"{config.api_key}",
    'X-RapidAPI-Host': "flight-fare-search.p.rapidapi.com"
}

conn.request("GET", "/v2/airport/departures?airportCode=ATL&date=2023-05-16&carrierCode=DL", headers=headers)

departures = conn.getresponse().read().decode('utf-8')

departures = json.loads(departures)

In [109]:
conn = http.client.HTTPSConnection("flight-fare-search.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': f"{config.api_key}",
    'X-RapidAPI-Host': "flight-fare-search.p.rapidapi.com"
}

conn.request("GET", "/v2/airport/arrivals?airportCode=ATL&date=2023-05-16&carrierCode=DL", headers=headers)

arrivals = conn.getresponse().read().decode('utf-8')

arrivals = json.loads(arrivals)

In [110]:
arrivals_data = arrivals['result']['flights']
departures_data = departures['result']['flights']


# normalize semi-structured JSON data into a flat table
df_arrivals = pd.json_normalize(arrivals_data)
df_departures = pd.json_normalize(departures_data)

In [111]:
df_departures['time'] = pd.to_datetime(df_departures['time'])
df_departures['date'] = df_departures['time'].dt.date

df_departures['time_EST'] = df_departures['time'].dt.tz_convert('US/Eastern')
df_departures['date_EST'] = df_departures['time_EST'].dt.date

df_departures['departure.time.time24'] = pd.to_datetime(df_departures['departure.time.time24'], format='%H:%M').dt.time
df_departures['arrival.time24'] = pd.to_datetime(df_departures['arrival.time24'], format='%H:%M').dt.time

In [112]:
df_arrivals['time'] = pd.to_datetime(df_arrivals['time'])
df_arrivals['date'] = df_arrivals['time'].dt.date

df_arrivals['time_EST'] = df_arrivals['time'].dt.tz_convert('US/Eastern')
df_arrivals['date_EST'] = df_arrivals['time_EST'].dt.date

df_arrivals['departure.time.time24'] = pd.to_datetime(df_arrivals['departure.time.time24'], format='%H:%M').dt.time
df_arrivals['arrival.time24'] = pd.to_datetime(df_arrivals['arrival.time24'], format='%H:%M').dt.time

In [113]:
df_arrivals

,time,operator,departure.time.timeAMPM,departure.time.time24,departure.airport.iata,departure.airport.city,arrival.timeAMPM,arrival.time24,carrier.code,carrier.name,carrier.flightNumber,date,time_EST,date_EST
0,2023-05-15 12:34:00+00:00,None,5:21AM,05:21:00,ATW,Appleton,8:34AM,08:34:00,DL,Delta Air Lines,1436,2023-05-15,2023-05-15 08:34:00-04:00,2023-05-15
1,2023-05-16 00:16:00+00:00,None,2:40PM,14:40:00,MEX,Mexico City,8:16PM,20:16:00,DL,Delta Air Lines,599,2023-05-16,2023-05-15 20:16:00-04:00,2023-05-15
2,2023-05-16 00:41:00+00:00,None,6:45PM,18:45:00,FLL,Fort Lauderdale,8:41PM,20:41:00,DL,Delta Air Lines,1546,2023-05-16,2023-05-15 20:41:00-04:00,2023-05-15
3,2023-05-16 01:15:00+00:00,None,4:00PM,16:00:00,QRO,Queretaro,9:15PM,21:15:00,DL,Delta Air Lines,9945,2023-05-16,2023-05-15 21:15:00-04:00,2023-05-15
4,2023-05-16 01:54:00+00:00,None,8:00PM,20:00:00,DCA,Washington,9:54PM,21:54:00,DL,Delta Air Lines,928,2023-05-16,2023-05-15 21:54:00-04:00,2023-05-15
5,2023-05-16 03:16:00+00:00,None,3:55PM,15:55:00,LAX,Los Angeles,11:16PM,23:16:00,DL,Delta Air Lines,873,2023-05-16,2023-05-15 23:16:00-04:00,2023-05-15
6,2023-05-16 03:37:00+00:00,None,8:05PM,20:05:00,MSP,Minneapolis,11:37PM,23:37:00,DL,Delta Air Lines,1549,2023-05-16,2023-05-15 23:37:00-04:00,2023-05-15
7,2023-05-16 04:06:00+00:00,None,5:05PM,17:05:00,LAS,Las Vegas,12:06AM,00:06:00,DL,Delta Air Lines,707,2023-05-16,2023-05-16 00:06:00-04:00,2023-05-16
8,2023-05-16 04:13:00+00:00,None,9:30PM,21:30:00,JFK,New York,12:13AM,00:13:00,DL,Delta Air Lines,1248,2023-05-16,2023-05-16 00:13:00-04:00,2023-05-16
9,2023-05-16 04:44:00+00:00,None,5:25PM,17:25:00,LAX,Los Angeles,12:44AM,00:44:00,DL,Delta Air Lines,803,2023-05-16,2023-05-16 00:44:00-04:00,2023-05-16


In [114]:
df_departures

,time,operator,departure.time.timeAMPM,departure.time.time24,departure.airport.iata,departure.airport.city,arrival.timeAMPM,arrival.time24,carrier.code,carrier.name,carrier.flightNumber,date,time_EST,date_EST
0,2023-05-16 01:16:00+00:00,None,9:16PM,21:16:00,PNS,Pensacola,9:29PM,21:29:00,DL,Delta Air Lines,2717,2023-05-16,2023-05-15 21:16:00-04:00,2023-05-15
1,2023-05-16 02:40:00+00:00,None,10:40PM,22:40:00,DEN,Denver,11:58PM,23:58:00,DL,Delta Air Lines,956,2023-05-16,2023-05-15 22:40:00-04:00,2023-05-15
2,2023-05-16 02:47:00+00:00,None,10:47PM,22:47:00,CHS,Charleston,11:59PM,23:59:00,DL,Delta Air Lines,2650,2023-05-16,2023-05-15 22:47:00-04:00,2023-05-15
3,2023-05-16 02:59:00+00:00,None,10:59PM,22:59:00,MDW,Chicago,11:52PM,23:52:00,DL,Delta Air Lines,1076,2023-05-16,2023-05-15 22:59:00-04:00,2023-05-15
4,2023-05-16 03:00:00+00:00,None,11:00PM,23:00:00,RDU,Raleigh/Durham,12:25AM,00:25:00,DL,Delta Air Lines,1431,2023-05-16,2023-05-15 23:00:00-04:00,2023-05-15
5,2023-05-16 03:00:00+00:00,None,11:00PM,23:00:00,VPS,Fort Walton Beach,11:13PM,23:13:00,DL,Delta Air Lines,2060,2023-05-16,2023-05-15 23:00:00-04:00,2023-05-15
6,2023-05-16 03:05:00+00:00,None,11:05PM,23:05:00,TYS,Knoxville,11:59PM,23:59:00,DL,Delta Air Lines,2876,2023-05-16,2023-05-15 23:05:00-04:00,2023-05-15
7,2023-05-16 03:28:00+00:00,None,11:28PM,23:28:00,MKE,Milwaukee,12:29AM,00:29:00,DL,Delta Air Lines,2970,2023-05-16,2023-05-15 23:28:00-04:00,2023-05-15
8,2023-05-16 03:35:00+00:00,None,11:35PM,23:35:00,FLL,Fort Lauderdale,1:26AM,01:26:00,DL,Delta Air Lines,2995,2023-05-16,2023-05-15 23:35:00-04:00,2023-05-15
9,2023-05-16 03:35:00+00:00,Operated by SkyWest Airlines on behalf of Delt...,11:35PM,23:35:00,AGS,Augusta,12:36AM,00:36:00,DL,Delta Air Lines,3609,2023-05-16,2023-05-15 23:35:00-04:00,2023-05-15


In [115]:
df_departures.rename(columns={'time':'Time_UTC',
                              'operator':'Operator',
                              'departure.time.timeAMPM':'Departure_Time_AM/PM',
                              'departure.time.time24':'Departure_Time_24',
                              'departure.airport.iata':'Arrival_Airport_IATA',
                              'departure.airport.city':'Arrival_Airport_City',
                              'arrival.timeAMPM':'Arrival_Time_AM/PM',
                              'arrival.time24':'Arrival_Time_24',
                              'carrier.code':'Airline_Code',
                              'carrier.name':'Airline_Name',
                              'carrier.flightNumber':'Airline_Flight_Number',
                              'date':'Date_UTC',
                              'time_EST':'Time_EST',
                              'date_EST':'Date_EST'}, inplace=True)

df_departures = df_departures[['Time_UTC', 'Date_UTC', 'Time_EST', 'Date_EST', 'Departure_Time_AM/PM', 'Departure_Time_24', 'Arrival_Airport_IATA', 'Arrival_Airport_City', 'Arrival_Time_AM/PM', 'Arrival_Time_24', 'Airline_Code', 'Airline_Name', 'Airline_Flight_Number']]

In [116]:
df_arrivals.rename(columns={'time':'Time_UTC',
                              'operator':'Operator',
                              'departure.time.timeAMPM':'Departure_Time_AM/PM',
                              'departure.time.time24':'Departure_Time_24',
                              'departure.airport.iata':'Departure_Airport_IATA',
                              'departure.airport.city':'Departure_Airport_City',
                              'arrival.timeAMPM':'Arrival_Time_AM/PM',
                              'arrival.time24':'Arrival_Time_24',
                              'carrier.code':'Airline_Code',
                              'carrier.name':'Airline_Name',
                              'carrier.flightNumber':'Airline_Flight_Number',
                              'date':'Date_UTC',
                              'time_EST':'Time_EST',
                              'date_EST':'Date_EST'}, inplace=True)

df_arrivals = df_arrivals[['Time_UTC', 'Date_UTC', 'Time_EST', 'Date_EST', 'Departure_Time_AM/PM', 'Departure_Time_24', 'Departure_Airport_IATA', 'Departure_Airport_City', 'Arrival_Time_AM/PM', 'Arrival_Time_24', 'Airline_Code', 'Airline_Name', 'Airline_Flight_Number']]

In [117]:
df_arrivals

,Time_UTC,Date_UTC,Time_EST,Date_EST,Departure_Time_AM/PM,Departure_Time_24,Departure_Airport_IATA,Departure_Airport_City,Arrival_Time_AM/PM,Arrival_Time_24,Airline_Code,Airline_Name,Airline_Flight_Number
0,2023-05-15 12:34:00+00:00,2023-05-15,2023-05-15 08:34:00-04:00,2023-05-15,5:21AM,05:21:00,ATW,Appleton,8:34AM,08:34:00,DL,Delta Air Lines,1436
1,2023-05-16 00:16:00+00:00,2023-05-16,2023-05-15 20:16:00-04:00,2023-05-15,2:40PM,14:40:00,MEX,Mexico City,8:16PM,20:16:00,DL,Delta Air Lines,599
2,2023-05-16 00:41:00+00:00,2023-05-16,2023-05-15 20:41:00-04:00,2023-05-15,6:45PM,18:45:00,FLL,Fort Lauderdale,8:41PM,20:41:00,DL,Delta Air Lines,1546
3,2023-05-16 01:15:00+00:00,2023-05-16,2023-05-15 21:15:00-04:00,2023-05-15,4:00PM,16:00:00,QRO,Queretaro,9:15PM,21:15:00,DL,Delta Air Lines,9945
4,2023-05-16 01:54:00+00:00,2023-05-16,2023-05-15 21:54:00-04:00,2023-05-15,8:00PM,20:00:00,DCA,Washington,9:54PM,21:54:00,DL,Delta Air Lines,928
5,2023-05-16 03:16:00+00:00,2023-05-16,2023-05-15 23:16:00-04:00,2023-05-15,3:55PM,15:55:00,LAX,Los Angeles,11:16PM,23:16:00,DL,Delta Air Lines,873
6,2023-05-16 03:37:00+00:00,2023-05-16,2023-05-15 23:37:00-04:00,2023-05-15,8:05PM,20:05:00,MSP,Minneapolis,11:37PM,23:37:00,DL,Delta Air Lines,1549
7,2023-05-16 04:06:00+00:00,2023-05-16,2023-05-16 00:06:00-04:00,2023-05-16,5:05PM,17:05:00,LAS,Las Vegas,12:06AM,00:06:00,DL,Delta Air Lines,707
8,2023-05-16 04:13:00+00:00,2023-05-16,2023-05-16 00:13:00-04:00,2023-05-16,9:30PM,21:30:00,JFK,New York,12:13AM,00:13:00,DL,Delta Air Lines,1248
9,2023-05-16 04:44:00+00:00,2023-05-16,2023-05-16 00:44:00-04:00,2023-05-16,5:25PM,17:25:00,LAX,Los Angeles,12:44AM,00:44:00,DL,Delta Air Lines,803
